In [1]:
!pip install transformers tensorflow

     |████████████████████████████████| 3.8 MB 5.5 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
     |████████████████████████████████| 6.5 MB 33.0 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 462 kB 47.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
import tensorflow as tf
import csv

In [4]:
text = []
labels = []

with open('../datasets/dataset.csv') as fp:
  reader = csv.reader(fp,delimiter=',')
  for row in reader:
    text.append(row[2])
    labels.append(row[3])
text = text[1:]
labels = labels[1:]
for i,val in enumerate(labels):
  labels[i] = int(labels[i])
tot_size = len(text)
train_ratio = 1
train_len=int(train_ratio*tot_size)
train_text = text[:train_len]
train_labels = labels[:train_len]




In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [6]:
train_encodings = tokenizer(train_text,truncation=True,padding=True)

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

In [8]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
model.fit(train_dataset.shuffle(len(train_text)).batch(16),
          epochs=5,
          batch_size=16)

Epoch 1/5
1503/1503 [==============================] - 759s 488ms/step - loss: 0.2980 - accuracy: 0.8721
Epoch 2/5
1503/1503 [==============================] - 734s 488ms/step - loss: 0.1673 - accuracy: 0.9376
Epoch 3/5
1503/1503 [==============================] - 733s 488ms/step - loss: 0.1139 - accuracy: 0.9579
Epoch 4/5
1503/1503 [==============================] - 734s 488ms/step - loss: 0.0978 - accuracy: 0.9646
Epoch 5/5
1503/1503 [==============================] - 727s 484ms/step - loss: 0.0760 - accuracy: 0.9747


In [11]:
model.save_pretrained('/models/roberta_pretrained')